In [1]:
import findspark
findspark.init()

In [2]:
from pyspark import SparkConf, SparkContext
import codecs

In [3]:
conf = SparkConf().setMaster("local").setAppName("MostPopularHero")
sc = SparkContext(conf = conf)

25/11/18 22:29:42 WARN Utils: Your hostname, turan-H110 resolves to a loopback address: 127.0.1.1; using 192.168.1.9 instead (on interface enp2s0)
25/11/18 22:29:42 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
25/11/18 22:29:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [4]:
def parseLine(line):
    heroIDs = line.split(" ")
    return heroIDs

In [5]:
def countCoOccurrences(line):
    elements = line.split()
    heroID = elements[0]
    connections = len(elements) - 1
    
    results = []
    results.append((int(heroID), connections))
    
    for co_hero in elements[1:]:
        results.append((int(co_hero), 1))
        
    return results

In [6]:
def loadHeroNames():
    heroNames = {}
    with codecs.open("/home/turan/SparkCourse/section_4/Marvel-Names", "r", 
                     encoding='ISO-8859-1', errors='ignore') as f:
        for line in f:
            fields = line.split('"')
            heroID = int(fields[0].strip())
            heroName = fields[1]
            heroNames[heroID] = heroName
    return heroNames

In [7]:
lines = sc.textFile("file:///home/turan/SparkCourse/section_4/Marvel-Graph")

In [8]:
nameDict = loadHeroNames()
broadcastedNames = sc.broadcast(nameDict)

In [9]:
parsedLines = lines.flatMap(parseLine)

In [10]:
pairings = lines.flatMap(countCoOccurrences)

In [11]:
totalConnections = pairings.reduceByKey(lambda x, y: x + y)

In [12]:
mostPopular = totalConnections.max(key=lambda x: x[1])

In [13]:
mostPopularName = broadcastedNames.value[mostPopular[0]]

In [14]:
print(f"En popüler süper kahraman: '{mostPopularName}' (ID: {mostPopular[0]})")
print(f"Toplam bağlantı sayısı: {mostPopular[1]}")

En popüler süper kahraman: 'CAPTAIN AMERICA' (ID: 859)
Toplam bağlantı sayısı: 3866
